# Using DeepAid for model interpretability 

Reference: DeepAID, Interpreting and Improving Deep Learning-Based Anomaly Detection in Security Applications [Github](https://github.com/dongtsi/DeepAID)

In [2]:
import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve
import torch

def validate_by_rmse(rmse_vec,thres,label):
    pred = np.asarray([0] * len(rmse_vec))
    idx = np.where(rmse_vec>thres)
    pred[idx] = 1
    cnf_matrix = confusion_matrix(label, pred)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    TPR = (TP/(TP+FN))[1]
    FPR = (FP/(FP+TN))[1]
    print("TPR:",TPR,"|FPR:",FPR)

    return pred

class Normalizer:
    def __init__(self, 
            dim, 
            normer="minmax",
            online_minmax=False): # whether fit_transform online (see Kitsune), *available only for normer="minmax"

        self.dim = dim # feature dimensionality
        self.normer = normer
        if self.normer == 'minmax':
            self.online_minmax = online_minmax
            self.norm_max = [-np.Inf] * self.dim
            self.norm_min = [np.Inf] * self.dim
        else:
            raise NotImplementedError # Implement other Normalizer here
        
    def fit_transform(self,train_feat):
        if self.normer == 'minmax':
            return self._minmax_fit_transform(train_feat)
        else:
            raise NotImplementedError # Implement other Normalizer here

    def transform(self,feat):
        if self.normer == 'minmax':
            return self._minmax_transform(feat)
        else:
            raise NotImplementedError # Implement other Normalizer here

    def restore(self,feat):
        if self.normer == 'minmax':
            return self._minmax_restore(feat)
        else:
            raise NotImplementedError # Implement other Normalizer here
        
    def _minmax_fit_transform(self,train_feat):
        if not self.online_minmax:
            self.norm_min = np.min(train_feat,axis=0)
            self.norm_max = np.max(train_feat,axis=0)
            norm_feat = (train_feat - self.norm_min) / (self.norm_max-self.norm_min+1e-10)
            return norm_feat
        else:
            norm_feat = []
            self.norm_max, self.norm_min = np.asarray(self.norm_max), np.asarray(self.norm_min)
            for i in range(len(train_feat)):
                x = train_feat[i]
                self.norm_max[x>self.norm_max] = x[x>self.norm_max]
                self.norm_min[x<self.norm_min] = x[x<self.norm_min]
                norm_feat.append((x - self.norm_min) / (self.norm_max-self.norm_min+1e-10))
            return np.asarray(norm_feat)

    def _minmax_transform(self, feat):
        norm_feat = (feat - self.norm_min) / (self.norm_max-self.norm_min+1e-10)
        return norm_feat

    def _minmax_restore(self, feat):
        denorm_feat = feat * (self.norm_max-self.norm_min+1e-10) + self.norm_min
        return denorm_feat
    
    # def _olminmax_fit_transform(self, train_feat):
    #     norm_feat = []
    #     for i in range(len(train_feat)):
    #         x = train_feat[i]
    #         self.norm_max[x>self.norm_max] = x[x>self.norm_max]
    #         self.norm_min[x<self.norm_min] = x[x<self.norm_min]
    #         norm_feat.append(x - self.norm_min) / (self.norm_max-self.norm_min+1e-10)
    #     return np.asarray(norm_feat)
    
    # def _olminmax_transform(self, feat):
    #     norm_feat = (feat - self.norm_min) / (self.norm_max-self.norm_min+1e-10)
    #     return norm_feat


""" Deeplog tools """
def deeplogtools_seqformat(model, abnormal_data, num_candidates, index=0):
    import keras.utils.np_utils as np_utils
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    X = abnormal_data.copy()
    y, X = X[:,-1], np_utils.to_categorical(X[:,:-1])
    Output = model(torch.from_numpy(X).type(torch.float).to(device))
    TP_idx = []
    for i in range(len(Output)):
        output = Output[i]
        label = y[i]
        predicted = torch.argsort(output)[-num_candidates:]
        if label not in predicted:
            TP_idx.append(i)
    seq_feat = np_utils.to_categorical(abnormal_data[TP_idx])
    feat = seq_feat[index]
    seq = torch.from_numpy(feat[:-1,:]).to(device)
    label = torch.tensor(np.argmax(feat[-1])).unsqueeze(0).to(device)
    return seq,label, abnormal_data[TP_idx][index]

""" Multi LSTM tools """
def multiLSTM_seqformat(test_feat, seq_len = 5, index=0):
    import more_itertools

    X_test = more_itertools.windowed(test_feat[:,:],n=seq_len,step=1)
    X_test = np.asarray(list(X_test))
    y_test = np.asarray(test_feat[seq_len-1:])

    # print("X_test:",X_test.shape,"y_test:",y_test.shape)
    i = index
    interp_feat = y_test[i]
    seq_feat = np.asarray([X_test[i]]) 
    # print("seq_feat:",seq_feat.shape,"interp_feat:",interp_feat.shape)

    return seq_feat, interp_feat

In [3]:
"""
This implementation of DeepLog is based on the open-source code at 
https://github.com/wuyifan18/DeepLog 
"""
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters 
num_classes = 28 # Fixed for this demo
window_size = 10 # Fixed for this demo 
num_layers = 2 
hidden_size = 64 
num_epochs = 300
batch_size = 2048
num_candidates = 9 

class LSTM_onehot(nn.Module):
    def __init__(self, hidden_size, num_layers, num_keys):
        super(LSTM_onehot, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(num_keys, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_keys)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out


def train_deeplog(input_seq, output_label):
    seq_dataset = TensorDataset(torch.tensor(input_seq, dtype=torch.long), torch.tensor(output_label))
    dataloader = DataLoader(seq_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    model = LSTM_onehot( hidden_size, num_layers, num_classes).to(device)
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    # Train the model
    start_time = time.time()
    total_step = len(dataloader)
    for epoch in range(num_epochs):  # Loop over the dataset multiple times
        train_loss = 0
        for step, (seq, label) in enumerate(dataloader):
            seq = seq.clone().detach().view(-1, window_size).to(device)
            seq = F.one_hot(seq,num_classes=num_classes).float()
            output = model(seq)
            
            loss = criterion(output, label.to(device))
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            train_loss += loss.item()
            optimizer.step()
        print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step))
        
    elapsed_time = time.time() - start_time
    print('elapsed_time: {:.3f}s'.format(elapsed_time))

    return model


def test_deeplog(model, test_normal_loader, test_abnormal_loader):
    model.eval()
    TP = 0
    FP = 0
    # Test the model
    start_time = time.time()
    with torch.no_grad():
        for line in test_normal_loader:
            for i in range(len(line) - window_size):
                seq = line[i:i + window_size]
                label = line[i + window_size]
                seq = torch.tensor(seq, dtype=torch.long).view(-1, window_size).to(device)
                seq = F.one_hot(seq,num_classes=num_classes).float()
                label = torch.tensor(label).view(-1).to(device)
                output = model(seq)
                predicted = torch.argsort(output, 1)[0][-num_candidates:]
                if label not in predicted:
                    FP += 1
                    break

    with torch.no_grad():
        for line in test_abnormal_loader:
            for i in range(len(line) - window_size):
                seq = line[i:i + window_size]
                label = line[i + window_size]
                if label == -1:
                    TP += 1
                    break
                seq = torch.tensor(seq, dtype=torch.long).view(-1, window_size).to(device)
                seq = F.one_hot(seq,num_classes=num_classes).float()
                label = torch.tensor(label).view(-1).to(device)
                output = model(seq)
                predicted = torch.argsort(output, 1)[0][-num_candidates:]
                if label not in predicted:
                    TP += 1
                    break

    elapsed_time = time.time() - start_time
    print('elapsed_time: {:.3f}s'.format(elapsed_time))
    # Compute precision, recall and F1-measure
    FN = len(test_abnormal_loader) - TP
    P = 100 * TP / (TP + FP)
    R = 100 * TP / (TP + FN)
    F1 = 2 * P * R / (P + R)
    print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
    print('Finished Predicting')

In [4]:
import torch
from deeplog import *

In [5]:
from deeplog import LSTM_onehot
import torch
model = torch.load('LSTM_onehot.pth.tar')

In [6]:
"""Step 2: Find an anomaly you are interested in"""
abnormal_data = np.load('abnormal_data.npy')
idx = 100
seq, label, anomaly_timeseries = deeplogtools_seqformat(model, abnormal_data, num_candidates=9, index=idx)

2022-10-25 10:28:37.242228: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-25 10:28:37.369062: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-25 10:28:37.369076: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-25 10:28:37.391596: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-25 10:28:37.865297: W tensorflow/stream_executor/platform/de

In [19]:
seq 

tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
  

In [20]:
label

tensor([9], device='cuda:0')

In [21]:
anomaly_timeseries

array([ 4, 10,  9, 13,  6,  7, 10, 13,  6, 10,  9])

In [25]:
"""Step 3: Create a DeepAID Interpreter"""

from timeseries_onehot import UniTimeseriesAID
feature_desc = np.load('log_key_meanning.npy') # feature_description
my_interpreter = UniTimeseriesAID(model, feature_desc=feature_desc, class_num=28)

Successfully Initialize <Univariate Timeseries Interptreter> for Model <LSTM_onehot>


In [26]:
"""Step 4: Interpret your anomaly and show the result"""
interpretation = my_interpreter(seq, label)
my_interpreter.show_table(anomaly_timeseries, interpretation)


Visualize Interpretation (Table View)
+------+-------------------------------------+-------+------+-------------------------------------+
| Ano. |               Meaning               | Diff. | Ref. |               Meaning*              |
+------+-------------------------------------+-------+------+-------------------------------------+
|  4   |      Receiving blk* src&dest:*      |       |  4   |      Receiving blk* src&dest:*      |
|  10  |  PktResponder* for blk* terminating |       |  10  |  PktResponder* for blk* terminating |
|  9   |       PktResponder* Exception       |       |  9   |       PktResponder* Exception       |
|  13  |  Exception in receiveBlock for blk* |       |  13  |  Exception in receiveBlock for blk* |
|  6   |   writeBlock* received exception*   |       |  6   |   writeBlock* received exception*   |
|  7   | PktResponder* for blk* Interrupted. |       |  7   | PktResponder* for blk* Interrupted. |
|  10  |  PktResponder* for blk* terminating |       |  10  |

/home/db1702/PhD Research/Explainability/deepaid/timeseries_onehot.py:61: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss_accuracy = Bound(self.bound_thres-Logit(out)[label[0]])
/home/db1702/PhD Research/Explainability/deepaid/timeseries_onehot.py:74: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if torch.max(Logit(out)).cpu().data > self.pos_thres:
/home/db1702/PhD Research/Explainability/deepaid/timeseries_onehot.py:77: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  IDX2.append(torch.argmax(Logit(out)).cpu().data.numpy().tolist())


## Second experiment

In [7]:
"""Step 2: Find an anomaly you are interested in"""
abnormal_data = np.load('abnormal_data.npy')
idx = 10
seq, label, anomaly_timeseries = deeplogtools_seqformat(model, abnormal_data, num_candidates=9, index=idx)

In [8]:
seq 

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
  

In [9]:
label

tensor([4], device='cuda:0')

In [10]:
anomaly_timeseries

array([10,  8, 25, 17, 24,  4, 25, 16, 24, 17,  4])

In [11]:
"""Step 3: Create a DeepAID Interpreter"""

from timeseries_onehot import UniTimeseriesAID
feature_desc = np.load('log_key_meanning.npy') # feature_description
my_interpreter = UniTimeseriesAID(model, feature_desc=feature_desc, class_num=28)

Successfully Initialize <Univariate Timeseries Interptreter> for Model <LSTM_onehot>


In [12]:
"""Step 4: Interpret your anomaly and show the result"""
interpretation = my_interpreter(seq, label)
my_interpreter.show_table(anomaly_timeseries, interpretation)


Visualize Interpretation (Table View)
+------+--------------------------------------+-------+------+--------------------------------------+
| Ano. |               Meaning                | Diff. | Ref. |               Meaning*               |
+------+--------------------------------------+-------+------+--------------------------------------+
|  10  |  PktResponder* for blk* terminating  |       |  10  |  PktResponder* for blk* terminating  |
|  8   |      Received blk* size* from*       |       |  8   |      Received blk* size* from*       |
|  25  |  NS.addStoredBlock: blkMap updated   |       |  25  |  NS.addStoredBlock: blkMap updated   |
|  17  | Starting thread to transfer blk* to* |       |  17  | Starting thread to transfer blk* to* |
|  24  |        ask* to replicate* to*        |       |  24  |        ask* to replicate* to*        |
|  4   |      Receiving blk* src&dest:*       |       |  4   |      Receiving blk* src&dest:*       |
|  25  |  NS.addStoredBlock: blkMap updated

/home/db1702/PhD Research/Explainability/deepaid/timeseries_onehot.py:61: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss_accuracy = Bound(self.bound_thres-Logit(out)[label[0]])
/home/db1702/PhD Research/Explainability/deepaid/timeseries_onehot.py:74: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if torch.max(Logit(out)).cpu().data > self.pos_thres:
/home/db1702/PhD Research/Explainability/deepaid/timeseries_onehot.py:77: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  IDX2.append(torch.argmax(Logit(out)).cpu().data.numpy().tolist())


In [15]:
## Third
"""Step 2: Find an anomaly you are interested in"""
abnormal_data = np.load('abnormal_data.npy')
idx = 1000
seq, label, anomaly_timeseries = deeplogtools_seqformat(model, abnormal_data, num_candidates=9, index=idx)
seq

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
  

In [16]:
label

tensor([8], device='cuda:0')

In [17]:
"""Step 3: Create a DeepAID Interpreter"""

from timeseries_onehot import UniTimeseriesAID
feature_desc = np.load('log_key_meanning.npy') # feature_description
my_interpreter = UniTimeseriesAID(model, feature_desc=feature_desc, class_num=28)
interpretation = my_interpreter(seq, label)
my_interpreter.show_table(anomaly_timeseries, interpretation)

Successfully Initialize <Univariate Timeseries Interptreter> for Model <LSTM_onehot>

Visualize Interpretation (Table View)
+------+------------------------------------+-------+------+------------------------------------+
| Ano. |              Meaning               | Diff. | Ref. |              Meaning*              |
+------+------------------------------------+-------+------+------------------------------------+
|  10  | PktResponder* for blk* terminating |       |  10  | PktResponder* for blk* terminating |
|  9   |      PktResponder* Exception       |       |  9   |      PktResponder* Exception       |
|  14  |  Changing blk file offset of blk*  |       |  14  |  Changing blk file offset of blk*  |
|  0   |   Adding an already existing blk   |       |  0   |   Adding an already existing blk   |
|  10  | PktResponder* for blk* terminating |       |  10  | PktResponder* for blk* terminating |
|  8   |     Received blk* size* from*      |       |  8   |     Received blk* size* from*  

In [18]:
#Fourth
## 
"""Step 2: Find an anomaly you are interested in"""
abnormal_data = np.load('abnormal_data.npy')
idx = 155
seq, label, anomaly_timeseries = deeplogtools_seqformat(model, abnormal_data, num_candidates=9, index=idx)
seq

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
  

In [19]:
label

tensor([19], device='cuda:0')

In [20]:
"""Step 3: Create a DeepAID Interpreter"""

from timeseries_onehot import UniTimeseriesAID
feature_desc = np.load('log_key_meanning.npy') # feature_description
my_interpreter = UniTimeseriesAID(model, feature_desc=feature_desc, class_num=28)
interpretation = my_interpreter(seq, label)
my_interpreter.show_table(anomaly_timeseries, interpretation)

Successfully Initialize <Univariate Timeseries Interptreter> for Model <LSTM_onehot>

Visualize Interpretation (Table View)
+------+----------------------------------------+-------+------+--------------------------------------+
| Ano. |                Meaning                 | Diff. | Ref. |               Meaning*               |
+------+----------------------------------------+-------+------+--------------------------------------+
|  15  |          Transmitted blk* to*          |       |  15  |         Transmitted blk* to*         |
|  25  |   NS.addStoredBlock: blkMap updated    |       |  25  |  NS.addStoredBlock: blkMap updated   |
|  25  |   NS.addStoredBlock: blkMap updated    |       |  25  |  NS.addStoredBlock: blkMap updated   |
|  20  |          Deleting blk* file*           |       |  20  |         Deleting blk* file*          |
|  22  |  NS.delete is added to invalidSet of*  |       |  22  | NS.delete is added to invalidSet of* |
|  22  |  NS.delete is added to invalidSet o